In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from General_Functions import long_horizon_ret
from Main import settings, features, pf_set
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, process_return_data, wealth_func, load_and_filter_market_returns_test
import numpy as np

In [2]:
def check_nan(df):
    nan_count = df.isna().sum()  # Antal NaN per kolonne
    nan_percentage = (nan_count / len(df)) * 100  # Procentandel af NaN per kolonne
    
    nan_df = pd.DataFrame({
        'Missing Values': nan_count,
        'Percentage (%)': nan_percentage
    }).sort_values(by='Missing Values', ascending=False)
    
    print(f"Total missing values in DataFrame: {df.isna().sum().sum()} ({df.isna().sum().sum() / df.size * 100:.2f}%)")
    
    return nan_df

## Initial plots of data

### Quick preprosseing steps

In [3]:
data = pd.read_csv('data_test/monthly_preprocessed_test.csv') 

In [4]:
data.head()

,id,eom,eom_ret,ret_ld1,tr_ld1,horizon,ret_ld2
0,10104,2010-01-31,2010-02-28,0.042934,0.042934,1,NaN
1,10104,2010-02-28,2010-03-31,0.007960,0.007960,1,NaN
2,10104,2010-03-31,2010-04-30,-0.127589,-0.127489,1,NaN
3,10104,2010-04-30,2010-05-31,-0.049307,-0.049207,1,NaN
4,10104,2010-05-31,2010-06-30,0.103772,0.103872,1,NaN


In [5]:
data.tail()

,id,eom,eom_ret,ret_ld1,tr_ld1,horizon,ret_ld2
57443,92655,2023-07-31,2023-08-31,NaN,0.062004,2,0.057504
57444,92655,2023-08-31,2023-09-30,NaN,0.032584,2,0.028084
57445,92655,2023-09-30,2023-10-31,NaN,-0.044489,2,-0.048789
57446,92655,2023-10-31,2023-11-30,NaN,0.004700,2,0.000000
57447,92655,2023-11-30,2023-12-31,NaN,0.004400,2,0.000000


In [6]:
print(data_filtered.dtypes)

NameError: name 'data_filtered' is not defined

In [ ]:
# Konverter 'eom_ret' til datetime
data["eom_ret"] = pd.to_datetime(data["eom_ret"])

# Filtrer data for horizon == 1
filtered_data = data[data["horizon"] == 1]

In [ ]:
# Vælg de første 5 unikke id'er
unique_ids = filtered_data["id"].unique()[:5]

# Initialiser plot
plt.figure(figsize=(12, 6))

# Beregn og plot akkumulerede afkast for hver id
for uid in unique_ids:
    subset = filtered_data[filtered_data["id"] == uid].sort_values("eom_ret")
    subset["cum_ret"] = (1 + subset["ret_ld1"]).cumprod() - 1  # Akkumuleret afkast
    plt.plot(subset["eom_ret"], subset["cum_ret"], label=f"ID {uid}")

# Tilpas plot
plt.xlabel("Dato")
plt.ylabel("Akkumuleret Afkast")
plt.title("Akkumuleret Afkast for de første 5 unikke ID'er")
plt.legend()
plt.grid()

# Vis plot
plt.show()

## Plots of every stock

In [ ]:
# Filtrer data for horizon == 1
filtered_data = data[data["horizon"] == 1]

# Initialiser plot
plt.figure(figsize=(14, 7))

# Loop gennem hver unik id og plot akkumuleret afkast
for uid in filtered_data["id"].unique():
    subset = filtered_data[filtered_data["id"] == uid].sort_values("eom")
    subset["cum_ret"] = (1 + subset["ret_ld1"]).cumprod() - 1  # Akkumuleret afkast
    plt.plot(subset["eom"], subset["cum_ret"], alpha=0.6)  # Tilføj gennemsigtighed for synlighed

# Tilpas plot
plt.xlabel("Dato")
plt.ylabel("Akkumuleret Afkast")
plt.title("Akkumuleret Afkast for alle ID'er")
plt.grid()
plt.show()


### Adding tickers

#### Plotting with tickers and names:

In [7]:
# Indlæs ID til Ticker mapping
ticker_map = pd.read_csv("Data/ticker_and_id.csv")  # Indlæs filen
ticker_dict = dict(zip(ticker_map["PERMNO"], ticker_map["TICKER"]))  # Omdan til dictionary

# Vælg de første 5 unikke id'er
unique_ids = filtered_data["id"].unique()[:5]

# Initialiser plot
plt.figure(figsize=(12, 6))

# Beregn og plot akkumulerede afkast for hver id
for uid in unique_ids:
    subset = filtered_data[filtered_data["id"] == uid].sort_values("eom_ret")
    subset["cum_ret"] = (1 + subset["ret_ld1"]).cumprod() - 1  # Akkumuleret afkast
    
    # Hent ticker fra mapping (brug ID hvis ingen match)
    ticker_label = ticker_dict.get(uid, f"ID {uid}")
    
    # Plot
    plt.plot(subset["eom_ret"], subset["cum_ret"], label=ticker_label)

# Tilpas plot
plt.xlabel("Dato")
plt.ylabel("Akkumuleret Afkast")
plt.title("Akkumuleret Afkast for de første 5 unikke Tickere")
plt.legend()
plt.grid()

# Vis plot
plt.show()

NameError: name 'filtered_data' is not defined

In [8]:
# Indlæs ID til Ticker mapping
ticker_map = pd.read_csv("Data/ticker_and_id.csv")  # Indlæs filen
ticker_dict = dict(zip(ticker_map["PERMNO"], ticker_map["COMNAM"]))  # Omdan til dictionary

# Vælg de første 5 unikke id'er
unique_ids = filtered_data["id"].unique()[:5]

# Initialiser plot
plt.figure(figsize=(12, 6))

# Beregn og plot akkumulerede afkast for hver id
for uid in unique_ids:
    subset = filtered_data[filtered_data["id"] == uid].sort_values("eom_ret")
    subset["cum_ret"] = (1 + subset["ret_ld1"]).cumprod() - 1  # Akkumuleret afkast
    
    # Hent ticker fra mapping (brug ID hvis ingen match)
    ticker_label = ticker_dict.get(uid, f"ID {uid}")
    
    # Plot
    plt.plot(subset["eom_ret"], subset["cum_ret"], label=ticker_label)

# Tilpas plot
plt.xlabel("Dato")
plt.ylabel("Akkumuleret Afkast")
plt.title("Akkumuleret Afkast for de første 5 unikke Tickere")
plt.legend()
plt.grid()

# Vis plot
plt.show()

NameError: name 'filtered_data' is not defined

## load and prepare data

In [13]:
file_path_usa_test = "./data_test/usa_test.parquet"  #This is with rvol_252d
#file_path_usa = "./Data/usa.parquet"  #This is with rvol_252d
data = pd.read_parquet(file_path_usa_test, engine="pyarrow")
#data2 = pd.read_parquet(file_path_usa, engine="pyarrow")
print(data.shape)
#print(data2.shape)

(28713, 166)


In [27]:
print(f"Features list length: {len(features)}")


Features list length: 115


In [16]:
features

['age',
 'aliq_at',
 'aliq_mat',
 'ami_126d',
 'at_be',
 'at_gr1',
 'at_me',
 'at_turnover',
 'be_gr1a',
 'be_me',
 'beta_60m',
 'beta_dimson_21d',
 'betabab_1260d',
 'betadown_252d',
 'bev_mev',
 'bidaskhl_21d',
 'capx_gr1',
 'cash_at',
 'chcsho_12m',
 'coa_gr1a',
 'col_gr1a',
 'cop_at',
 'cop_atl1',
 'corr_1260d',
 'coskew_21d',
 'cowc_gr1a',
 'dbnetis_at',
 'debt_me',
 'div12m_me',
 'dolvol_126d',
 'dolvol_var_126d',
 'ebit_bev',
 'ebit_sale',
 'ebitda_mev',
 'emp_gr1',
 'eq_dur',
 'eqnpo_12m',
 'fcf_me',
 'fnl_gr1a',
 'gp_at',
 'gp_atl1',
 'ival_me',
 'inv_gr1',
 'inv_gr1a',
 'iskew_capm_21d',
 'iskew_ff3_21d',
 'ivol_capm_21d',
 'ivol_capm_252d',
 'ivol_ff3_21d',
 'kz_index',
 'lnoa_gr1a',
 'lti_gr1a',
 'market_equity',
 'mispricing_mgmt',
 'mispricing_perf',
 'ncoa_gr1a',
 'ncol_gr1a',
 'netdebt_me',
 'nfna_gr1a',
 'ni_be',
 'ni_me',
 'nncoa_gr1a',
 'noa_at',
 'noa_gr1a',
 'o_score',
 'oaccruals_at',
 'oaccruals_ni',
 'ocf_at',
 'ocf_at_chg1',
 'ocf_me',
 'op_at',
 'op_atl1',
 'o

In [28]:
file_path_usa_test = "./data_test/usa_test.parquet"  #This is with rvol_252d
# Liste over kolonner, vi gerne vil have
# Vi har fjernet "ff49" og "crsp_exchcd", for vi har dem ikke i vores data
desired_cols = ["id", "eom", "sic", "size_grp", "me", "rvol_252d", "dolvol_126d"] + features #I R får vi 166 features i vores datasæt, tror det

print(desired_cols)
print(f"desired_cols list length: {len(desired_cols)}")
#går galt her

# Tjek hvilke kolonner der faktisk findes i Parquet-filen
actual_cols = pd.read_parquet(file_path_usa_test, engine="pyarrow").columns
cols_to_load = [col for col in desired_cols if col in actual_cols]

if not cols_to_load:
    raise ValueError("Ingen af de ønskede kolonner findes i Parquet-filen.")

# Indlæs kun de kolonner, der faktisk findes
chars = pd.read_parquet(file_path_usa_test, engine="pyarrow", columns=desired_cols)
# Filtrér observationer
chars = chars[chars["id"] <= 99999]

# Konverter datoformat for 'eom'
chars["eom"] = pd.to_datetime(chars["eom"], errors="coerce")

['id', 'eom', 'sic', 'size_grp', 'me', 'rvol_252d', 'dolvol_126d', 'age', 'aliq_at', 'aliq_mat', 'ami_126d', 'at_be', 'at_gr1', 'at_me', 'at_turnover', 'be_gr1a', 'be_me', 'beta_60m', 'beta_dimson_21d', 'betabab_1260d', 'betadown_252d', 'bev_mev', 'bidaskhl_21d', 'capx_gr1', 'cash_at', 'chcsho_12m', 'coa_gr1a', 'col_gr1a', 'cop_at', 'cop_atl1', 'corr_1260d', 'coskew_21d', 'cowc_gr1a', 'dbnetis_at', 'debt_me', 'div12m_me', 'dolvol_126d', 'dolvol_var_126d', 'ebit_bev', 'ebit_sale', 'ebitda_mev', 'emp_gr1', 'eq_dur', 'eqnpo_12m', 'fcf_me', 'fnl_gr1a', 'gp_at', 'gp_atl1', 'ival_me', 'inv_gr1', 'inv_gr1a', 'iskew_capm_21d', 'iskew_ff3_21d', 'ivol_capm_21d', 'ivol_capm_252d', 'ivol_ff3_21d', 'kz_index', 'lnoa_gr1a', 'lti_gr1a', 'market_equity', 'mispricing_mgmt', 'mispricing_perf', 'ncoa_gr1a', 'ncol_gr1a', 'netdebt_me', 'nfna_gr1a', 'ni_be', 'ni_me', 'nncoa_gr1a', 'noa_at', 'noa_gr1a', 'o_score', 'oaccruals_at', 'oaccruals_ni', 'ocf_at', 'ocf_at_chg1', 'ocf_me', 'op_at', 'op_atl1', 'ope_be'

In [12]:
print(chars.shape)

(28713, 122)


In [29]:

# Tilføj kolonnen 'lambda'
pi = settings["pi"]
#chars["dolvol"] = chars["dolvol_126d"]  # Kopiér dolvol_126d til dolvol
#chars["lambda"] = 2 / chars["dolvol"] * pi  # Beregn lambda
#chars["rvol_m"] = chars["rvol_252d"] * np.sqrt(21)  # Beregn rvol_m



chars = chars.loc[:, ~chars.columns.duplicated()]
    
# Kontrollér for nødvendige kolonner
required_cols = ["dolvol_126d", "rvol_252d"]
for col in required_cols:
    if col not in chars.columns:
        raise KeyError(f"Kolonnen '{col}' mangler i DataFrame.")

# Debugging: Tjek for null-værdier
for col in required_cols:
    if chars[col].isnull().any():
        raise ValueError(f"Kolonnen '{col}' indeholder manglende værdier.")
    
# Debugging: Tjek for 0-værdier i 'dolvol_126d'
if (chars["dolvol_126d"] == 0).any():
    raise ValueError("Kolonnen 'dolvol_126d' indeholder værdier lig 0, hvilket vil føre til divisionsfejl.")

# Beregn afledte kolonner
print("Beregner 'dolvol', 'lambda' og 'rvol_m'...")
chars["dolvol"] = chars["dolvol_126d"]  # Kopiér dolvol_126d til dolvol
chars["lambda"] = 2 / chars["dolvol"] * pi  # Beregn lambda
chars["rvol_m"] = chars["rvol_252d"] * np.sqrt(21)  # Beregn rvol_m

print(chars)


Beregner 'dolvol', 'lambda' og 'rvol_m'...
          id        eom   sic size_grp            me  rvol_252d   dolvol_126d  \
0      10104 2010-01-31  7372     mega  115558.73052   0.019544  6.798938e+08   
1      10104 2010-02-28  7372     mega  123619.74809   0.018949  7.094750e+08   
2      10104 2010-03-31  7372     mega  129040.82501   0.015241  6.982804e+08   
3      10104 2010-04-30  7372     mega  130101.18774   0.014489  6.802223e+08   
4      10104 2010-05-31  7372     mega  113436.81847   0.015136  7.368972e+08   
...      ...        ...   ...      ...           ...        ...           ...   
28708  92655 2023-07-31  6324     mega  471446.66929   0.014563  1.732788e+09   
28709  92655 2023-08-31  6324     mega  441458.42446   0.014580  1.619178e+09   
28710  92655 2023-09-30  6324     mega  467033.72021   0.014428  1.619647e+09   
28711  92655 2023-10-31  6324     mega  496091.90354   0.014193  1.578173e+09   
28712  92655 2023-11-30  6324     mega  511455.75015   0.013393  1

In [30]:
#Vi skal nu bruge data_ret_ld1: 
file_path_world_ret="./data_test/world_ret_test.csv"
risk_free_path="./data_test/risk_free_test.csv"
data_ret_ld1 = process_return_data(file_path_world_ret, risk_free_path)

All missing excludes 0.60% of the observations


In [31]:
# Merge chars ind i data_ret_ld1 baseret på id og eom
#chars = data_ret_ld1.merge(chars, on=["id", "eom"], how="left")
chars = chars.merge(data_ret_ld1, on=["id", "eom"], how="left")
print(chars)

          id        eom   sic size_grp            me  rvol_252d   dolvol_126d  \
0      10104 2010-01-31  7372     mega  115558.73052   0.019544  6.798938e+08   
1      10104 2010-02-28  7372     mega  123619.74809   0.018949  7.094750e+08   
2      10104 2010-03-31  7372     mega  129040.82501   0.015241  6.982804e+08   
3      10104 2010-04-30  7372     mega  130101.18774   0.014489  6.802223e+08   
4      10104 2010-05-31  7372     mega  113436.81847   0.015136  7.368972e+08   
...      ...        ...   ...      ...           ...        ...           ...   
28708  92655 2023-07-31  6324     mega  471446.66929   0.014563  1.732788e+09   
28709  92655 2023-08-31  6324     mega  441458.42446   0.014580  1.619178e+09   
28710  92655 2023-09-30  6324     mega  467033.72021   0.014428  1.619647e+09   
28711  92655 2023-10-31  6324     mega  496091.90354   0.014193  1.578173e+09   
28712  92655 2023-11-30  6324     mega  511455.75015   0.013393  1.621881e+09   

       age   aliq_at  aliq_

In [32]:
print(data_ret_ld1.dtypes)

id                  int64
eom        datetime64[ns]
tr_ld0            float64
eom_ret    datetime64[ns]
ret_ld1           float64
tr_ld1            float64
dtype: object


In [33]:
print(chars.dtypes)

id                   int64
eom         datetime64[ns]
sic                 object
size_grp            object
me                 float64
                 ...      
rvol_m             float64
tr_ld0             float64
eom_ret     datetime64[ns]
ret_ld1            float64
tr_ld1             float64
Length: 127, dtype: object


In [34]:
market_path="./data_test/market_returns_test.csv"
market = load_and_filter_market_returns_test(market_path)
print(market.head())
risk_free = process_risk_free_rate(risk_free_path)
print(risk_free.head())
#wealth_end = 1e+10
wealth_end = pf_set["wealth"]
end = settings["split"]["test_end"]
#end = "2023-11-30"

wealth = wealth_func(wealth_end, end, market, risk_free)
print(wealth.head())
print(wealth.tail())

Filen er indlæst og filtreret succesfuldt.
          eom  mkt_vw_exc
0  2010-01-31   -0.038043
1  2010-02-28    0.034614
2  2010-03-31    0.064495
3  2010-04-30    0.019148
4  2010-05-31   -0.081046
         eom      rf
0 2010-01-31  0.0000
1 2010-02-28  0.0000
2 2010-03-31  0.0001
3 2010-04-30  0.0001
4 2010-05-31  0.0001
         eom        wealth    mu_ld1
0 2009-12-31  1.468331e+09 -0.038043
1 2010-01-31  1.414519e+09  0.034614
2 2010-02-28  1.465237e+09  0.064595
3 2010-03-31  1.566419e+09  0.019248
4 2010-04-30  1.597162e+09 -0.080946
           eom        wealth    mu_ld1
163 2023-07-31  9.975058e+09 -0.019997
164 2023-08-31  9.779494e+09 -0.048173
165 2023-09-30  9.330037e+09 -0.029172
166 2023-10-31  9.065579e+09  0.093442
167 2023-11-30  1.000000e+10       NaN


In [35]:
# Forskyd 'eom' med en måned i wealth DataFrame
wealth["eom"] = wealth["eom"] + pd.offsets.MonthEnd(1)
print(wealth)

# Behold kun relevante kolonner fra wealth
wealth_subset = wealth[["eom", "mu_ld1"]].rename(columns={"mu_ld1": "mu_ld0"})

# Merge wealth ind i chars baseret på 'eom'
chars = chars.merge(wealth_subset, on="eom", how="left")
print(chars)

           eom        wealth    mu_ld1
0   2010-01-31  1.468331e+09 -0.038043
1   2010-02-28  1.414519e+09  0.034614
2   2010-03-31  1.465237e+09  0.064595
3   2010-04-30  1.566419e+09  0.019248
4   2010-05-31  1.597162e+09 -0.080946
..         ...           ...       ...
163 2023-08-31  9.975058e+09 -0.019997
164 2023-09-30  9.779494e+09 -0.048173
165 2023-10-31  9.330037e+09 -0.029172
166 2023-11-30  9.065579e+09  0.093442
167 2023-12-31  1.000000e+10       NaN

[168 rows x 3 columns]
          id        eom   sic size_grp            me  rvol_252d   dolvol_126d  \
0      10104 2010-01-31  7372     mega  115558.73052   0.019544  6.798938e+08   
1      10104 2010-02-28  7372     mega  123619.74809   0.018949  7.094750e+08   
2      10104 2010-03-31  7372     mega  129040.82501   0.015241  6.982804e+08   
3      10104 2010-04-30  7372     mega  130101.18774   0.014489  6.802223e+08   
4      10104 2010-05-31  7372     mega  113436.81847   0.015136  7.368972e+08   
...      ...        ..

In [36]:
specific_id = 10104  # Erstat med det ID, du vil filtrere
filtered_chars = chars[chars["id"] == specific_id]

print(filtered_chars)  # Udskriver alle kolonner for det valgte ID

        id        eom   sic size_grp            me  rvol_252d   dolvol_126d  \
0    10104 2010-01-31  7372     mega  115558.73052   0.019544  6.798938e+08   
1    10104 2010-02-28  7372     mega  123619.74809   0.018949  7.094750e+08   
2    10104 2010-03-31  7372     mega  129040.82501   0.015241  6.982804e+08   
3    10104 2010-04-30  7372     mega  130101.18774   0.014489  6.802223e+08   
4    10104 2010-05-31  7372     mega  113436.81847   0.015136  7.368972e+08   
..     ...        ...   ...      ...           ...        ...           ...   
162  10104 2023-07-31  7379     mega  318192.59168   0.016165  9.220588e+08   
163  10104 2023-08-31  7379     mega  326769.63935   0.016484  9.632093e+08   
164  10104 2023-09-30  7379     mega  290154.70090   0.018170  1.134544e+09   
165  10104 2023-10-31  7379     mega  283251.48258   0.017441  1.147283e+09   
166  10104 2023-11-30  7379     mega  318342.88245   0.017289  1.138572e+09   

     age   aliq_at  aliq_mat  ...  zero_trades_21d 

## Prepare matrix of future monthly returns

In [56]:
file_path_world_ret_test="./data_test/world_ret_test.csv"
file_path_risk_free_test="./data_test/risk_free_test.csv"
monthly = pd.read_csv(file_path_world_ret_test, usecols=["excntry", "id", "eom", "ret_exc"], dtype={"eom": str})

# Filtrér kun USA og CRSP-observationer
monthly = monthly[(monthly["excntry"] == "USA") & (monthly["id"] <= 99999)]

# Konverter 'eom' fra string til datetime
monthly["eom"] = pd.to_datetime(monthly["eom"]) #monthly["eom"] = pd.to_datetime(monthly["eom"], format="%Y%m%d"), this is already in the right dataformat
#print(monthly)
#print(monthly.dtypes)
print(monthly)

horizon_K = settings["pf"]["hps"]["m1"]["K"]

# Anvend long_horizon_ret med de relevante parametre
data_ret = long_horizon_ret(monthly, h=horizon_K, impute="zero")
data_ret = data_ret.drop(columns=["start", "end", "merge_date"])


# Vis de første par rækker for at bekræfte resultatet
print(data_ret)
#print(data_ret)
nan_stats = check_nan(data_ret)
print(nan_stats)

          id excntry        eom   ret_exc
0      76076     USA 2010-01-31 -0.061432
1      11955     USA 2010-01-31 -0.052084
2      77702     USA 2010-01-31 -0.055102
3      77274     USA 2010-01-31  0.115525
4      76614     USA 2010-01-31  0.102536
...      ...     ...        ...       ...
28719  73139     USA 2023-11-30  0.092201
28720  59459     USA 2023-11-30  0.074291
28721  71563     USA 2023-11-30  0.147198
28722  72726     USA 2023-11-30  0.122297
28723  60442     USA 2023-11-30  0.165839

[28724 rows x 4 columns]
          id        eom   ret_ld1   ret_ld2   ret_ld3   ret_ld4   ret_ld5  \
0      10104 2010-01-31  0.068941  0.042934  0.007960 -0.127589 -0.049307   
1      10104 2010-02-28  0.042934  0.007960 -0.127589 -0.049307  0.103772   
2      10104 2010-03-31  0.007960 -0.127589 -0.049307  0.103772 -0.076057   
3      10104 2010-04-30 -0.127589 -0.049307  0.103772 -0.076057  0.228989   
4      10104 2010-05-31 -0.049307  0.103772 -0.076057  0.228989  0.095977   
...     

In [57]:
# Behold kun nødvendige kolonner og beregn eom_ret
data_ret_ld1 = data_ret[["id", "eom", "ret_ld1"]].copy()

# Beregn eom_ret: sidste dag i næste måned
data_ret_ld1["eom_ret"] = data_ret_ld1["eom"] + pd.DateOffset(months=1)
data_ret_ld1["eom_ret"] = data_ret_ld1["eom_ret"] + MonthEnd(0)
# Omarranger kolonnerne, så 'ret_ld1' kommer sidst
data_ret_ld1 = data_ret_ld1[["id", "eom", "eom_ret", "ret_ld1"]]
print(data_ret_ld1)

          id        eom    eom_ret   ret_ld1
0      10104 2010-01-31 2010-02-28  0.068941
1      10104 2010-02-28 2010-03-31  0.042934
2      10104 2010-03-31 2010-04-30  0.007960
3      10104 2010-04-30 2010-05-31 -0.127589
4      10104 2010-05-31 2010-06-30 -0.049307
...      ...        ...        ...       ...
28719  92655 2023-07-31 2023-08-31 -0.063368
28720  92655 2023-08-31 2023-09-30  0.057611
28721  92655 2023-09-30 2023-10-31  0.057504
28722  92655 2023-10-31 2023-11-30  0.028084
28723  92655 2023-11-30 2023-12-31  0.000000

[28724 rows x 4 columns]


In [58]:
data_ret_ld1 = data_ret_ld1.merge(risk_free, on="eom", how="left")
print(data_ret_ld1)

          id        eom    eom_ret   ret_ld1      rf
0      10104 2010-01-31 2010-02-28  0.068941  0.0000
1      10104 2010-02-28 2010-03-31  0.042934  0.0000
2      10104 2010-03-31 2010-04-30  0.007960  0.0001
3      10104 2010-04-30 2010-05-31 -0.127589  0.0001
4      10104 2010-05-31 2010-06-30 -0.049307  0.0001
...      ...        ...        ...       ...     ...
28719  92655 2023-07-31 2023-08-31 -0.063368  0.0045
28720  92655 2023-08-31 2023-09-30  0.057611  0.0045
28721  92655 2023-09-30 2023-10-31  0.057504  0.0043
28722  92655 2023-10-31 2023-11-30  0.028084  0.0047
28723  92655 2023-11-30 2023-12-31  0.000000  0.0044

[28724 rows x 5 columns]


In [59]:
#Beregn total afkast: excess return + rf
data_ret_ld1["tr_ld1"] = data_ret_ld1["ret_ld1"] + data_ret_ld1["rf"]
print(data_ret_ld1)

          id        eom    eom_ret   ret_ld1      rf    tr_ld1
0      10104 2010-01-31 2010-02-28  0.068941  0.0000  0.068941
1      10104 2010-02-28 2010-03-31  0.042934  0.0000  0.042934
2      10104 2010-03-31 2010-04-30  0.007960  0.0001  0.008060
3      10104 2010-04-30 2010-05-31 -0.127589  0.0001 -0.127489
4      10104 2010-05-31 2010-06-30 -0.049307  0.0001 -0.049207
...      ...        ...        ...       ...     ...       ...
28719  92655 2023-07-31 2023-08-31 -0.063368  0.0045 -0.058868
28720  92655 2023-08-31 2023-09-30  0.057611  0.0045  0.062111
28721  92655 2023-09-30 2023-10-31  0.057504  0.0043  0.061804
28722  92655 2023-10-31 2023-11-30  0.028084  0.0047  0.032784
28723  92655 2023-11-30 2023-12-31  0.000000  0.0044  0.004400

[28724 rows x 6 columns]


In [60]:
# Fjern 'rf' kolonnen efter brug
data_ret_ld1.drop(columns=["rf"], inplace=True)
print(data_ret_ld1)

          id        eom    eom_ret   ret_ld1    tr_ld1
0      10104 2010-01-31 2010-02-28  0.068941  0.068941
1      10104 2010-02-28 2010-03-31  0.042934  0.042934
2      10104 2010-03-31 2010-04-30  0.007960  0.008060
3      10104 2010-04-30 2010-05-31 -0.127589 -0.127489
4      10104 2010-05-31 2010-06-30 -0.049307 -0.049207
...      ...        ...        ...       ...       ...
28719  92655 2023-07-31 2023-08-31 -0.063368 -0.058868
28720  92655 2023-08-31 2023-09-30  0.057611  0.062111
28721  92655 2023-09-30 2023-10-31  0.057504  0.061804
28722  92655 2023-10-31 2023-11-30  0.028084  0.032784
28723  92655 2023-11-30 2023-12-31  0.000000  0.004400

[28724 rows x 5 columns]


In [61]:
# Opret en kopi af data_ret_ld1 med justeret eom
data_ret_ld1_shifted = data_ret_ld1[["id", "eom", "tr_ld1"]].copy()

# Flyt eom til næste måneds sidste dag
data_ret_ld1_shifted["eom"] = data_ret_ld1_shifted["eom"] + pd.DateOffset(months=1)
data_ret_ld1_shifted["eom"] = data_ret_ld1_shifted["eom"] + MonthEnd(0)

# Omdøb tr_ld1 til tr_ld0
data_ret_ld1_shifted.rename(columns={"tr_ld1": "tr_ld0"}, inplace=True)

# Merge for at inkludere tr_ld0 i data_ret_ld1
data_ret_ld1 = data_ret_ld1.merge(data_ret_ld1_shifted, on=["id", "eom"], how="left")

#Omarranger kolonner
data_ret_ld1 = data_ret_ld1[["id", "eom", "tr_ld0", "eom_ret", "ret_ld1", "tr_ld1"]]

# Fjern 'monthly' variablen fra hukommelsen
del monthly

# Vis de første rækker for at bekræfte resultatet
print(data_ret_ld1)

          id        eom    tr_ld0    eom_ret   ret_ld1    tr_ld1
0      10104 2010-01-31       NaN 2010-02-28  0.068941  0.068941
1      10104 2010-02-28  0.068941 2010-03-31  0.042934  0.042934
2      10104 2010-03-31  0.042934 2010-04-30  0.007960  0.008060
3      10104 2010-04-30  0.008060 2010-05-31 -0.127589 -0.127489
4      10104 2010-05-31 -0.127489 2010-06-30 -0.049307 -0.049207
...      ...        ...       ...        ...       ...       ...
28719  92655 2023-07-31  0.053142 2023-08-31 -0.063368 -0.058868
28720  92655 2023-08-31 -0.058868 2023-09-30  0.057611  0.062111
28721  92655 2023-09-30  0.062111 2023-10-31  0.057504  0.061804
28722  92655 2023-10-31  0.061804 2023-11-30  0.028084  0.032784
28723  92655 2023-11-30  0.032784 2023-12-31  0.000000  0.004400

[28724 rows x 6 columns]


## Lav monthly_ret_monthly.csv fil kun med vores test id'er

In [8]:
file_path_id_test="./data_test/top_5_percent_ids.csv"
file_path_world_ret="./Data/world_ret_monthly.csv"

In [9]:
world_ret = pd.read_csv(file_path_world_ret)
id_test = pd.read_csv(file_path_id_test)

#### Her filtreres kun for de unikke id'er for test-datasættet

In [15]:
# Filtrér world_ret for kun at inkludere id'er, der er i id_test
filtered_world_ret = world_ret[world_ret['id'].isin(id_test['id'])]

# Gem den filtrerede dataframe til en ny CSV-fil
filtered_world_ret.to_csv("filtered_world_ret.csv", index=False)

# Bekræftelse på filtrering
print(filtered_world_ret.head())
print(filtered_world_ret.shape)

#Start-dato-test: start_date = "2010-01-31   slut_dato-test: 
#start_date = "2010-01-31
#end_date = "2023-11-30"
unique_ids_count = filtered_world_ret['id'].nunique()
print(f"Antal unikke ID'er: {unique_ids_count}")

        id excntry       eom          me  ret  ret_local  ret_exc  source_crsp
350  10145     USA  19251231  248.292000  NaN        NaN      NaN            1
447  14277     USA  19251231    7.140000  NaN        NaN      NaN            1
456  13928     USA  19251231  111.367500  NaN        NaN      NaN            1
457  13901     USA  19251231    8.118438  NaN        NaN      NaN            1
458  13856     USA  19251231    4.956250  NaN        NaN      NaN            1
(102785, 8)
Antal unikke ID'er: 172


In [17]:
# Konverter 'eom' til datetime-format
filtered_world_ret['eom'] = pd.to_datetime(filtered_world_ret['eom'], format='%Y%m%d')

# Definer dato-interval
start_date = "2010-01-31"
end_date = "2023-11-30"

# Filtrér baseret på dato-intervallet
filtered_world_ret = filtered_world_ret[(filtered_world_ret['eom'] >= start_date) & (filtered_world_ret['eom'] <= end_date)]

# Gem den filtrerede dataframe til en ny CSV-fil
#filtered_world_ret.to_csv("filtered_world_ret_filtered_by_date.csv", index=False)

# Bekræftelse
print(filtered_world_ret.head())
print(filtered_world_ret.tail())
print(f"Shape efter dato-filter: {filtered_world_ret.shape}")

# Tæl unikke ID'er efter filtrering
unique_ids_count = filtered_world_ret['id'].nunique()
print(f"Antal unikke ID'er efter dato-filter: {unique_ids_count}")

             id excntry        eom             me       ret  ret_local  \
13273331  76076     USA 2010-01-31  128842.976060 -0.061404  -0.061404   
13273375  11955     USA 2010-01-31   15693.346226 -0.052056  -0.052056   
13273450  77702     USA 2010-01-31   16198.686681 -0.055074  -0.055074   
13273577  77274     USA 2010-01-31   43431.077722  0.115553   0.115553   
13273579  76614     USA 2010-01-31    2117.923708  0.102564   0.102564   

           ret_exc  source_crsp  
13273331 -0.061432            1  
13273375 -0.052084            1  
13273450 -0.055102            1  
13273577  0.115525            1  
13273579  0.102536            1  
             id excntry        eom             me       ret  ret_local  \
25021086  73139     USA 2023-11-30  112574.280250  0.096625   0.096625   
25021088  59459     USA 2023-11-30   41253.426265  0.078715   0.078715   
25021089  71563     USA 2023-11-30   42864.088706  0.151622   0.151622   
25021093  72726     USA 2023-11-30   22471.086786  0.12

## mothly full data (not ready yet)

In [4]:
# Indlæs data
monthly = pd.read_csv("Data/world_ret_monthly.csv", usecols=["excntry", "id", "eom", "ret_exc"], dtype={"eom": str})

# Filtrér kun USA og CRSP-observationer
monthly = monthly[(monthly["excntry"] == "USA") & (monthly["id"] <= 99999)]

# Konverter 'eom' fra string til datetime
monthly["eom"] = pd.to_datetime(monthly["eom"], format="%Y%m%d")

print(monthly)

             id excntry        eom   ret_exc
0         12589     USA 1925-12-31       NaN
1         14349     USA 1925-12-31       NaN
2         14461     USA 1925-12-31       NaN
3         11631     USA 1925-12-31       NaN
4         11287     USA 1925-12-31       NaN
...         ...     ...        ...       ...
25108332  11018     USA 2023-12-31  0.092401
25108333  10777     USA 2023-12-31 -0.037593
25108334  10696     USA 2023-12-31  0.012808
25108335  11006     USA 2023-12-31  0.177632
25108336  75517     USA 2023-12-31  0.111914

[5037353 rows x 4 columns]


In [5]:
horizon_K = settings["pf"]["hps"]["m1"]["K"]

# Anvend long_horizon_ret med de relevante parametre
data_ret = long_horizon_ret(monthly, h=horizon_K, impute="zero")

# Vis de første par rækker for at bekræfte resultatet
print(data_ret.head())

MemoryError: Unable to allocate 338. MiB for an array with shape (44305800, 1) and data type float64